In [147]:
import pandas as pd
import spacy
from scispacy.abbreviation import AbbreviationDetector

In [148]:
nlp = spacy.load("en_core_web_sm")

In [149]:
#Add the abbreviation detector to the pipeline
nlp.add_pipe("abbreviation_detector")

In [150]:
def resolve_abbreviation(text):
    doc = nlp(text)
    mapping = {}
    for abrv in doc._.abbreviations:
# Resolve the abbreviation to its full form
        mapping[str(abrv)]=str(abrv._.long_form)
    replaced_text = text
    for abrv, long_form in mapping.items():
        replaced_text = replaced_text.replace(abrv, long_form)
    return replaced_text

In [151]:
#Read the Excel file that has student Answers
Studentscript = pd.read_excel("StudentScript.xlsx", sheet_name="Sheet1")

#Read the Excel file that has the correct Answers
Memo = pd.read_excel("Memo.xlsx", sheet_name="Sheet1")

In [152]:
#Merge the two dataframes
Compare = pd.merge(Studentscript, Memo, on="QuestionNumber", suffixes=('_Student', '_Memo'))
#Compare['Mark']= Compare.apply(lambda row: 2 if row['Answers_Student'] == row['Answers_Memo'] else 0, axis=1)

In [153]:
#Identify Entities that are similar in both student and memo answers
def contains_named_entity_match(Answer_student, answer_memo):
    
    student_response = nlp(Answer_student)
    memo_response = nlp(answer_memo)
    
    student_entities = {ent.text for ent in student_response.ents}
    memo_entities = {ent.text for ent in memo_response.ents}
    
    return len(student_entities.intersection(memo_entities)) > 0

In [154]:
#Using Spacy(Open source NLP library) to compare student answers with memo answers
def spacy_similarity(Answer_student,Answer_memo,threshold=0.6):
   student_response = nlp(Answer_student)
   memo_response = nlp(Answer_memo)
   similarity = student_response.similarity(memo_response)
   return 2 if similarity >= threshold else 0 

In [155]:
#Combine the marking logic
def combined_marking(Answer_student, Answer_memo, threshold=0.6):
    
    resolved_student = resolve_abbreviation(Answer_student)
    resolved_memo = resolve_abbreviation(Answer_memo)
    
    if contains_named_entity_match(Answer_student, Answer_memo):
        return 2
    else:
        return spacy_similarity(Answer_student, Answer_memo, threshold)

In [156]:
#For the semantic evaluation using spacy(An open-sourse NLP library)
#The evaluation kicks in if the response has greater than 7 characters
def smart_marking(row):
    if len(str(row['Answers_Student'])) > 7:
        return combined_marking(row['Answers_Student'], row['Answers_Memo'])
    else:
        return 2 if row['Answers_Student'] == row['Answers_Memo'] else 0

In [ ]:
#Compare['Mark']= Compare.apply(lambda row: 2 if row['Answers_Student'] == row['Answers_Memo'] else 0, axis=1)
Compare['Mark'] = Compare.apply(smart_marking, axis=1)

c:\Users\rafey\AppData\Local\Programs\Python\Python311\Lib\site-packages\scispacy\abbreviation.py:248: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)
C:\Users\rafey\AppData\Local\Temp\ipykernel_6892\888248231.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = student_response.similarity(memo_response)


,QuestionNumber,Answers_Student,Answers_Memo,Mark
0,1,Artificial intelligence is important because i...,Artificial Intelligence has a wide-ranging imp...,0
1,2,Imaginative and often exaggerated portrayals i...,Artificial Intelligence is perceived as includ...,0
2,3,Artificial intelligence is a discipline in com...,Artificial intelligence is a discipline in com...,2
3,4,A,A,2
4,5,B,A,0
5,6,The Capital City of South Africa is Pretoria,Pretoria,2
6,7,To highlight/show the rapid growth of Artifici...,To highlight/show the rapid growth of Artifici...,2
7,8,The words suggest that using Artificial Intell...,The words suggest that using Artificial Intell...,2
8,9,S A,South Africa,0


In [158]:
#Add up the total marks
total_marks = Compare['Mark'].sum()
#Update the Compare DataFrame with the total marks
Compare.loc['Total'] = ['','','',total_marks]
Compare

,QuestionNumber,Answers_Student,Answers_Memo,Mark
0,1,Artificial intelligence is important because i...,Artificial Intelligence has a wide-ranging imp...,0
1,2,Imaginative and often exaggerated portrayals i...,Artificial Intelligence is perceived as includ...,0
2,3,Artificial intelligence is a discipline in com...,Artificial intelligence is a discipline in com...,2
3,4,A,A,2
4,5,B,A,0
5,6,The Capital City of South Africa is Pretoria,Pretoria,2
6,7,To highlight/show the rapid growth of Artifici...,To highlight/show the rapid growth of Artifici...,2
7,8,The words suggest that using Artificial Intell...,The words suggest that using Artificial Intell...,2
8,9,S A,South Africa,0
Total,,,,10


In [159]:
#Save the results to an Excel file
Compare.to_excel("Marking.xlsx", index=False)